In [1]:
from quantdev.data import Databank, DatasetsHandler
from quantdev.backtest import *
from quantdev.trade import *
# from quantdev.trade import *
# from quantdev.analysis import *
# from quantdev.trade import PortfolioManager, Position, Portfolio

# from typing import Literal
# import pandas as pd
# import numpy as np

In [ ]:
## pending: use np.where to replace the loc in backtest, adding trading cost and slippage

In [110]:
DatasetsHandler().update_datasets()

## backtest

In [2]:
data = Databank()
roe = data['常續ROE'].to_factor().to_rank()
pbr = (-1*data['股價淨值比']).to_factor().to_rank()
mtm = (data['收盤價']*data['調整係數']).pct_change(60, fill_method=None).to_factor().to_rank()

roe_strategy = backtesting(roe.largest(10), 'QR', report=True)
pbr_strategy = backtesting(pbr.largest(10), 'QR', report=True)
mtm_strategy = backtesting(mtm.largest(10), 'QR', report=True)
# combined_strategy = backtesting((roe+pbr+mtm).largest(10), 'QR', report=True)

,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),15.19,9.75,4.69
Sharpe,0.74,0.61,0.37
Calmar,0.24,0.17,0.13
MDD(%),-64.47,-56.11,-36.26
Simple MDD(%),-47.55,-47.09,-23.96
Win Rate(%),54.32,54.80,51.58
Weekly Win(%),55.42,56.36,50.71
Monthly Win(%),62.30,62.70,50.82
Yearly Win(%),71.43,71.43,57.14
Win/Loss Ratio,0.92,0.92,1.00


,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),1.64,9.75,-8.68
Sharpe,0.18,0.61,-0.37
Calmar,0.02,0.17,-0.10
MDD(%),-82.39,-56.11,-87.82
Simple MDD(%),-97.65,-47.09,-148.07
Win Rate(%),51.52,54.80,48.26
Weekly Win(%),50.71,56.36,44.86
Monthly Win(%),51.64,62.70,43.85
Yearly Win(%),47.62,71.43,38.10
Win/Loss Ratio,0.95,0.92,1.01


,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),14.48,9.75,3.87
Sharpe,0.68,0.61,0.30
Calmar,0.25,0.17,0.09
MDD(%),-56.97,-56.11,-41.95
Simple MDD(%),-32.33,-47.09,-28.35
Win Rate(%),53.84,54.80,50.84
Weekly Win(%),55.61,56.36,51.46
Monthly Win(%),56.15,62.70,51.23
Yearly Win(%),71.43,71.43,42.86
Win/Loss Ratio,0.93,0.92,1.02


In [22]:
strategies = {
    'roe': (roe_strategy, 1),
    'pbr': (pbr_strategy, 1),
    'mtm': (mtm_strategy, 1),
}
multi_strategy = multi_backtesting(strategies, None)

,Strategy,Benchmark: TRTEJ,Excess Return,roe,pbr,mtm
CAGR(%),11.84,9.61,1.44,13.33,8.99,12.33
Sharpe,0.70,0.60,0.18,0.71,0.57,0.64
Calmar,0.17,0.17,0.04,0.21,0.12,0.18
MDD(%),-69.24,-56.11,-40.68,-63.81,-77.89,-69.65
Simple MDD(%),-57.65,-47.09,-36.33,-49.02,-73.98,-55.24
Win Rate(%),57.21,54.79,52.51,55.41,53.85,55.01
Weekly Win(%),58.06,56.36,52.31,56.08,53.06,56.27
Monthly Win(%),57.79,62.70,47.13,60.25,54.92,56.56
Yearly Win(%),66.67,71.43,57.14,71.43,66.67,66.67
Win/Loss Ratio,0.83,0.92,0.93,0.88,0.93,0.88


In [23]:
multi_strategy.tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'08d9addc-c316-4497-9f71-22d62f2c8c8e': {'version…

In [24]:
ROE = factor_analysis(roe)
ROE.tabs

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),3.17,8.35,9.00,4.50,8.03,13.99,13.12,14.07,15.54,16.31,12.44
Sharpe,0.27,0.55,0.58,0.34,0.53,0.85,0.80,0.85,0.91,0.90,1.25
Calmar,0.04,0.13,0.13,0.07,0.13,0.23,0.20,0.22,0.24,0.25,0.47
MDD(%),-74.97,-66.09,-67.38,-68.85,-63.28,-61.32,-64.22,-63.34,-63.69,-64.32,-26.40
Simple MDD(%),-88.03,-60.33,-60.48,-105.33,-64.74,-44.19,-51.99,-50.73,-50.37,-46.44,-17.30
Win Rate(%),57.05,57.83,57.89,57.81,58.09,59.02,59.34,58.54,58.44,57.79,52.13
Weekly Win(%),55.61,58.44,58.91,59.38,59.94,60.89,60.32,60.13,59.28,59.28,54.48
Monthly Win(%),51.64,55.74,56.56,56.15,59.84,60.66,61.48,65.57,64.75,64.75,62.70
Yearly Win(%),57.14,61.90,66.67,66.67,66.67,76.19,71.43,71.43,71.43,76.19,80.95
Win/Loss Ratio,0.76,0.78,0.78,0.75,0.77,0.79,0.77,0.80,0.81,0.83,1.11


BokehModel(combine_events=True, render_bundle={'docs_json': {'5095fe15-8cdd-4410-9d7f-41d11d36c1f3': {'version…

## SinoPac trade

In [5]:
money = 1000000
portfolio_by_money = {
    'roe strategy':(gen_position_info(roe_strategy).weight * money).to_dict(),
    'pbr strategy':(gen_position_info(pbr_strategy).weight * money).to_dict(),
    'mtm strategy':(gen_position_info(mtm_strategy).weight * money).to_dict(),
}

In [2]:
api = SinoPacAccount()
trader = SinoPacTrader(api)

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
portfolio.json loaded from /Users/jianrui/Desktop/Research/Quant/portfolio/sinopac_portfolio.json
portfolio.json matches inventories.


In [93]:
class SinoPacTrader:
    def __init__(self, api:SinoPacAccount, portfolio_path:str=None, price_tolerance:float=0.02, odd:bool=True):
        self.api = api
        self.price_tolerance = price_tolerance
        self.odd = odd

        # portfolio
        self.portfolio_path = portfolio_path or config.portfolio_path_config.get('sinopac_portfolio_path')
        self.portfolio = self.read_portfolio()
        self.check_portfolio_match(self.portfolio)
    
    def _update_status(self):
        return self.api.update_status(self.api.stock_account)

    # basic
    def list_inventories(self) -> List[sj.position.StockPosition]:
        return self.api.list_positions(self.api.stock_account, unit=sj.constant.Unit.Share)

    def list_orders(self) -> List[sj.order.Order]:
        return self.api.list_trades()

    def place_order(self, 
        stock_id:str, action:Literal['buy', 'sell'], intraday_odd:Literal[0, 1], 
        price:float, quantity:int, 
        price_type:Literal['limit', 'mkt']='limit', 
        order_type:Literal['fok', 'ioc', 'rod']='rod', 
        trade_type:Literal['cash', 'margin', 'short']='cash', 
        custom_field:str=None,
    ):
        param = {
            'action':{
                'buy':sj.constant.Action.Buy, 
                'sell':sj.constant.Action.Sell,
            },
            'intraday_odd':{
               0:sj.constant.StockOrderLot.Common, 
               1:sj.constant.StockOrderLot.IntradayOdd, 
            #  2:sj.constant.StockOrderLot.Fixing, 
            #  3:sj.constant.StockOrderLot.Odd, 
            },
            'price_type':{
                'limit':sj.constant.StockPriceType.LMT,
                'mkt':sj.constant.StockPriceType.MKT,
            },
            'order_type':{
                'fok':sj.constant.OrderType.FOK,
                'ioc':sj.constant.OrderType.IOC,
                'rod':sj.constant.OrderType.ROD,
            },
            'trade_type':{
                'cash':'Cash',
                'margin':'MarginTrading',
                'short':'ShortSelling',
            },
        }
        contract = self.api.Contracts.Stocks[stock_id]
        order = self.api.Order(
            quantity = quantity/1000 if intraday_odd==0 else quantity,
            price = price,
            action = param['action'][action],
            price_type = param['price_type'][price_type],
            order_type = param['order_type'][order_type],
            order_cond = param['trade_type'][trade_type],
            order_lot = param['intraday_odd'][intraday_odd],
            custom_field = custom_field,
            account=self.api.stock_account,
        )

        print(f'{action} {quantity} of {stock_id} at {price} for strategy {custom_field}')
        return self.api.place_order(contract, order)

    def cancel_order(self, order:sj.order.Order):
        return self.api.cancel_order(order)
   
    def get_bank_balance(self)-> float:
        return self.api.account_balance().acc_balance
    
    def get_settlement(self, as_df:bool=False):
        settlements = self.api.settlements(self.api.stock_account)
        if not as_df:
            return settlements
        else:
            return pd.DataFrame([s.__dict__ for s in settlements]).set_index("T")

    # portfolio
    def check_portfolio_match(self, portfolio:dict):
        agg_json_portfolio = {}
        for p in portfolio['portfolio'].values():
            for stock_id, quantity in p.items():
                if stock_id not in agg_json_portfolio:
                    agg_json_portfolio[stock_id] = quantity
                else:
                    agg_json_portfolio[stock_id] += quantity

        agg_inv_portfolio = {i.code:i.quantity * (-1 if i.direction == 'Sell' else 1)  for i in self.list_inventories()}
        diff_portfolio = {}
        for k, v in agg_json_portfolio.items():
            if k not in agg_inv_portfolio:
                diff_portfolio[k] = v
            else:
                diff_portfolio[k] = v - agg_inv_portfolio[k]
        diff_portfolio = {k:v for k, v in diff_portfolio.items() if v != 0}
        if diff_portfolio=={}:
            print('portfolio.json matches inventories.')
        else:
            print(f'portfolio.json does not match inventories: {diff_portfolio}!')

    def update_portfolio(self):
        current_portfolio = self.portfolio['portfolio']
        last_update_time = dt.datetime.strptime(self.portfolio['update_time'], '%Y-%m-%d %H:%M:%S')
        traded_orders={
            (order.order.custom_field if order.order.custom_field else 'BETA', order.contract.code):\
            max(order.contract.unit, 1) * \
            (-1 if order.order.action=='Sell' else 1) * \
            sum([deal.quantity for deal in order.status.deals if dt.datetime.fromtimestamp(deal.ts) > last_update_time])
            for order in self.list_orders() if \
            (order.status.status in ['Filled', 'Filling'])
        }

        for k, v in traded_orders.items():
            if current_portfolio.get(k[0]).get(k[1]):
                current_portfolio[k[0]][k[1]] += v
        self.portfolio['portfolio'] = current_portfolio
        self.portfolio['update_time'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    def read_portfolio(self):
        path = self.portfolio_path
        print(f'portfolio.json loaded from {path}')
        return json.load(open(f'{path}', 'r'))

    def write_portfolio(self):
        self.update_portfolio()
        path = self.portfolio_path
        with open(f'{path}', 'w') as f:
            json.dump(self.portfolio, f)
        print(f'portfolio.json saved to {path}')

    # trade
    def settlement_is_enough(self, diff_portfolio:Dict[str, List[Position]]) -> bool:
        try:
            bank_balance = self.get_bank_balance()
            settlement = sum([int(i['amount']) for i in self.get_settlement()])
            portfolio_money = sum([sum([p.round_price*p.round_qty+p.odd_price*p.odd_qty for p in positions]) for positions in diff_portfolio.values()])
            
            return (bank_balance + settlement - portfolio_money) >= 0
        except Exception as e:
            print(f'error when checking settlement: {e}')
            return True
        
    def check_portfolio_difference(self,new_portfolio:Dict[str, List[Dict[str, Union[int, float]]]], by:Literal['money', 'quantity']='money'):
        difference_portfolio = {}
        if by == 'money':
            for strategy, positions in new_portfolio.items():
                difference_portfolio[strategy] = []
                current_strategy = self.portfolio['portfolio'].get(strategy, {})
                
                def process_money(stock_id, money):
                    inv_money = current_strategy.get(stock_id, 0) * next((inv.price for inv in trader.list_inventories() if inv.code == stock_id), 0)
                    position = Position().from_money(stock_id, money - inv_money)
                    if not position.empty:
                        return position
                    return None
                
                with ThreadPoolExecutor() as executor:
                    futures = [executor.submit(process_money, stock_id, money) for stock_id, money in positions.items()]
                    for future in as_completed(futures):
                        position = future.result()
                        if position:
                            difference_portfolio[strategy].append(position)   
        elif by == 'quantity':
            for strategy, positions in new_portfolio.items():
                difference_portfolio[strategy] = []
                current_strategy = self.portfolio['portfolio'].get(strategy, {})
                
                def process_quantity(stock_id, quantity):
                    inv_quantity = current_strategy.get(stock_id, 0)
                    position = Position().from_quantity(stock_id, quantity - inv_quantity)
                    if not position.empty:
                        return position
                    return None
                
                with ThreadPoolExecutor() as executor:
                    futures = [executor.submit(process_quantity, stock_id, quantity) for stock_id, quantity in positions.items()]
                    for future in as_completed(futures):
                        position = future.result()
                        if position:
                            difference_portfolio[strategy].append(position)
        return difference_portfolio
    
    def trade_position(self, position:Position, strategy_name:str=None):
        action = 'buy' if ((position.round_qty+position.odd_qty)>0) else 'sell'
        # place round order
        if position.round_qty!=0:
            self.place_order(
                stock_id=position.stock_id,
                action=action,
                intraday_odd=0,
                price=position.round_price,
                quantity=abs(position.round_qty),
                custom_field=strategy_name,
            )
        
        # place odd order
        if position.odd_qty!=0:
            self.place_order(
                stock_id=position.stock_id,
                action=action,
                intraday_odd=1,
                price=position.odd_price,
                quantity=abs(position.odd_qty),
                custom_field=strategy_name,
            )

    def rebalance(self, new_portfolio:Dict[str, List[Dict[str, Union[int, float]]]], by:Literal['money', 'quantity']='money', freq:int=1, ignore_mkt_open:bool=False, ignore_settlement:bool=False):
        while True:
            # machine time
            print(f"Current time: {dt.datetime.now().time().strftime('%H:%M:%S')}")
            if (not ignore_mkt_open) and (not mkt_is_open()):
                print("Market is closed.")
                break
            
            # cancel all orders
            self._update_status()
            for order in self.list_orders():
                if order['status']['status'] not in ['Filled', 'Cancelled', 'Failed']:
                    self.cancel_order(order)

            # update portfolio
            self.update_portfolio()

            # check difference
            diff_portfolio = self.check_portfolio_difference(new_portfolio, by=by)

            if (not ignore_settlement) and (not self.settlement_is_enough(diff_portfolio)):
                print("Settlement is not enough for position!")
                break
            
            if not all(v==[] for v in diff_portfolio.values()):
                # trade
                for strategy_name, positions in diff_portfolio.items():
                    for position in positions:
                        self.trade_position(position, strategy_name)
            else:
                print('Portfolio is all filled.')
                self.write_portfolio()
                break
            
            time.sleep(60*freq)

{'portfolio': {'TW3F': {'2937': 6000},
  'SUE01': {'4946': 4498},
  'BETA': {'0050': 6143}},
 'update_time': '2025-05-05 12:28:25'}

In [112]:
check_portfolio_difference(new_portfolio={'TW3F':{}}, by='quantity')

{'TW3F': []}

In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def check_portfolio_difference(new_portfolio:Dict[str, List[Dict[str, Union[int, float]]]], by:Literal['money', 'quantity']='money'):
    difference_portfolio = {}
    if by == 'money':
        for strategy, positions in new_portfolio.items():
            difference_portfolio[strategy] = []
            current_strategy = trader.portfolio['portfolio'].get(strategy, {})
            
            def process_money(stock_id, money):
                inv_money = current_strategy.get(stock_id, 0) * next((inv.price for inv in trader.list_inventories() if inv.code == stock_id), 0)
                position = Position().from_money(stock_id, money - inv_money)
                if not position.empty:
                    return position
                return None
            
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(process_money, stock_id, money) for stock_id, money in positions.items()]
                for future in as_completed(futures):
                    position = future.result()
                    if position:
                        difference_portfolio[strategy].append(position)
                        
    elif by == 'quantity':
        for strategy, positions in new_portfolio.items():
            difference_portfolio[strategy] = []
            current_strategy = trader.portfolio['portfolio'].get(strategy, {})
            
            def process_quantity(stock_id, quantity):
                inv_quantity = current_strategy.get(stock_id, 0)
                position = Position().from_quantity(stock_id, quantity - inv_quantity)
                if not position.empty:
                    return position
                return None
            
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(process_quantity, stock_id, quantity) for stock_id, quantity in positions.items()]
                for future in as_completed(futures):
                    position = future.result()
                    if position:
                        difference_portfolio[strategy].append(position)
    return difference_portfolio

def trade_position(position:Position, strategy_name:str=None):
    
    action = 'buy' if ((position.round_qty+position.odd_qty)>0) else 'sell'
    # place round order
    if position.round_qty!=0:
        trader.place_order(
            stock_id=position.stock_id,
            action=action,
            intraday_odd=0,
            price=position.round_price,
            quantity=abs(position.round_qty),
            custom_field=strategy_name,
        )
    
    # place odd order
    if position.odd_qty!=0:
        self.place_order(
            stock_id=position.stock_id,
            action=action,
            intraday_odd=1,
            price=position.odd_price,
            quantity=abs(position.odd_qty),
            custom_field=strategy_name,
        )

def rebalance(portfolio:Dict[str, List[Dict[str, Union[int, float]]]], by:Literal['money', 'quantity']='money', freq:int=1):

    while True:
        # cancel all orders
        trader._update_status()
        for order in trader.list_orders():
            if order['status']['status'] not in ['Filled', 'Cancelled', 'Failed']:
                trader.cancel_order(order)

        # update portfolio
        trader.update_portfolio()

        # check difference
        diff_portfolio = check_portfolio_difference(portfolio, by=by)
        
        if not all(v==[] for v in diff_portfolio.values()):
            # trade
            for strategy, positions in diff_portfolio.items():
                for position in positions:
                    trader.trade_position(position, strategy)
        else:
            print('Portfolio is all filled.')
            trader.update_portfolio()
            trader.write_portfolio()
            break
        
        time.sleep(60*freq)



In [7]:
Position().from_quantity('2937', -2000)

{'stock_id': '2937', 'round_price': 43.85, 'odd_price': 43.05, 'round_qty': -2000.0, 'odd_qty': 0}

In [114]:
# max(order.contract.unit, 1) * -1 if order.order.action=='Sell' else 1
# trader._update_status()
# current_portfolio = trader.portfolio['portfolio']
# last_update_time = dt.datetime.strptime(trader.portfolio['update_time'], '%Y-%m-%d %H:%M:%S')
traded_orders={
    (order.order.custom_field if order.order.custom_field else 'BETA', order.contract.code):\
    max(order.contract.unit, 1) * \
    (-1 if order.order.action=='Sell' else 1) * \
    sum([deal.quantity for deal in order.status.deals if dt.datetime.fromtimestamp(deal.ts) > last_update_time])
    for order in trader.list_orders()
}
traded_orders
# for k, v in traded_orders.items():
#     if current_portfolio.get(k[0]).get(k[1]):
#         current_portfolio[k[0]][k[1]] += v
# trader.portfolio['portfolio'] = current_portfolio
# trader.portfolio['update_time'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

{('TW3F', '2937'): 0, ('BETA', '0050'): 0}

In [116]:
trader.write_portfolio()

portfolio.json saved to /Users/jianrui/Desktop/Research/Quant/portfolio/sinopac_portfolio.json


In [120]:
path = config.portfolio_path_config.get('fubon_portfolio_path')
try:
    json.load(open(f'{path}', 'r'))
except FileNotFoundError:
    print(f"Creating new portfolio file at {path}")
    default_portfolio = {
        'portfolio': {},
        'update_time': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    with open(f'{path}', 'w') as f:
        json.dump(default_portfolio, f)
    return default_portfolio

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jianrui/Desktop/Research/Quant/portfolio/fubon_portfolio.json'

SDK NOTICE Mon May 05 14:47:18.080 2025 solClient.c:11120                    (30c093000) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/A127945492/0505/021435/054841/101.10.82.94', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 172.20.10.8:53945'
SDK NOTICE Mon May 05 14:47:18.084 2025 solClientOS.c:4658                   (30c093000) TCP connection failure for fd 82, error = Network is unreachable (51)
SDK NOTICE Mon May 05 14:47:18.084 2025 solClient.c:5028                     (30c093000) Session '(c0,s1)_sinopac', could not open transport for tcp_TxRx
SDK NOTICE Mon May 05 14:47:18.084 2025 solClient.c:10165                    (30c093000) Session '(c0,s1)_sinopac' error attempting transport connection, client name 'PYAPI/A127945492/0505/021435/054841/101.10.82.94', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local

Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/A127945492/0505/021435/054841/101.10.82.94', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP  | Event: Session reconnecting


SDK NOTICE Mon May 05 14:47:21.152 2025 solClientOS.c:4658                   (30c093000) TCP connection failure for fd 83, error = Network is unreachable (51)
SDK NOTICE Mon May 05 14:47:21.152 2025 solClient.c:5028                     (30c093000) Session '(c0,s1)_sinopac', could not open transport for tcp_TxRx
SDK NOTICE Mon May 05 14:47:21.152 2025 solClient.c:10165                    (30c093000) Session '(c0,s1)_sinopac' error attempting transport connection, client name 'PYAPI/A127945492/0505/021435/054841/101.10.82.94', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 0.0.0.0:58318'
SDK NOTICE Mon May 05 14:47:21.152 2025 solClient.c:10320                    (30c093000) Protocol or communication error when attempting to login for session '(c0,s1)_sinopac'; are session HOST and PORT correct? client name 'PYAPI/A127945492/0505/021435/054841/101.10.82.94', VPN name 'sinopac', peer host '210.59.255.161:80' address '

Session Down.
Response Code: 0 | Event Code: 1 | Info: TCP connection failure for fd 83, error = Network is unreachable (51) | Event: Session down


In [48]:
[order for order in trader.list_orders() if \
 (order.status.order_datetime > dt.datetime.strptime(trader.portfolio['update_time'], '%Y-%m-%d %H:%M:%S')) &\
 (order.status.status in ['Filled', 'Filling'])]

[Trade(contract=Stock(exchange=<Exchange.OTC: 'OTC'>, code='2937', symbol='OTC2937', name='集雅社', category='38', unit=1000, limit_up=48.4, limit_down=39.6, reference=44.0, update_date='2025/05/05'), order=Order(action=<Action.Sell: 'Sell'>, price=43.85, quantity=2, id='d43be698', seqno='295826', ordno='Y02VQ', account=Account(account_type=<AccountType.Stock: 'S'>, person_id='A127945492', broker_id='9A98', account_id='0284473', signed=True), custom_field='TW3F', price_type=<StockPriceType.LMT: 'LMT'>, order_type=<OrderType.ROD: 'ROD'>), status=OrderStatus(id='d43be698', status=<Status.Filled: 'Filled'>, status_code='00', web_id='137', order_datetime=datetime.datetime(2025, 5, 5, 10, 16, 29), deal_quantity=2, deals=[Deal(seq='807237', price=43.85, quantity=1, ts=1746413226.0), Deal(seq='807762', price=43.85, quantity=1, ts=1746413236.0)])),
 Trade(contract=Contract(security_type=<SecurityType.Stock: 'STK'>, exchange=<Exchange.TSE: 'TSE'>, code='0050'), order=Order(action=<Action.Sell: 'Se

In [22]:
# Convert Unix timestamp to datetime
timestamp = 1746413226.0
datetime_obj = dt.datetime.fromtimestamp(timestamp)
print(f"Unix timestamp {timestamp} converts to datetime: {datetime_obj}")


Unix timestamp 1746413226.0 converts to datetime: 2025-05-05 10:47:06


In [25]:
trader.portfolio

{'portfolio': {'TW3F': {'2937': 10000},
  'SUE01': {'4946': 4498},
  'BETA': {'0050': 6257}},
 'update_time': '2025-05-04 11:36:01'}

OrderState.StockOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '4f0d7c7a', 'seqno': '37987T', 'ordno': 'Y03GE', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A98', 'account_id': '0284473', 'signed': True}, 'action': 'Sell', 'price': 170.6, 'quantity': 57, 'order_type': 'ROD', 'price_type': 'LMT', 'order_cond': 'Cash', 'order_lot': 'IntradayOdd', 'custom_field': ''}, 'status': {'id': '4f0d7c7a', 'exchange_ts': 1746413770.486, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 57, 'web_id': '267'}, 'contract': {'security_type': 'STK', 'exchange': 'TSE', 'code': '0050', 'symbol': '', 'name': '', 'currency': 'TWD'}}
OrderState.StockDeal {'trade_id': '4f0d7c7a', 'seqno': '37987T', 'ordno': 'Y03GE', 'exchange_seq': '000239', 'broker_id': '9A98', 'account_id': '0284473', 'action': 'Sell', 'code': '0050', 'order_cond': 'Cash', 'order_lot': 'IntradayOdd', 'price': 170.7, 'quantity': 8, 'web_id': '267', 'custom_field': '', 't

In [18]:
import datetime as dt

# Convert string to datetime object for comparison
update_time = dt.datetime.strptime(trader.portfolio['update_time'], '%Y-%m-%d %H:%M:%S')
# current_time = dt.datetime(2025, 5, 5, 10, 16, 29)

# Compare the two datetime objects
# print(f"Current time: {current_time}")
# print(f"Update time: {update_time}")
# print(f"Is current time later than update time? {current_time > update_time}")

In [31]:
check_portfolio_difference({'TW3F':{'2937': 0}}, by='money')

462500.0


{'TW3F': [{'stock_id': '2937', 'round_price': 43.1, 'round_qty': -10000.0, 'odd_price': 43.2, 'odd_qty': -729.0}]}

In [37]:
all(v==[] for v in {'TW3F':[], 'SUE01':[]}.values())

True

In [27]:
(46.25-44)*10000

22500.0

In [25]:
729*43.2

31492.800000000003

In [31]:
current_portfolio = trader.portfolio
inventories = trader.list_inventories()

{'portfolio': {'TW3F': [{'2937': 100000}],
  'SUE01': {'4946': 4498},
  'BETA': {'0050': 6257}},
 'update_time': '2025-05-04 11:36:01'}

In [11]:
stock_id = '29371'


0

In [37]:
{'TW3F':{'2937': 100000}}.keys()

dict_keys(['TW3F'])

In [36]:
check_portfolio_difference({'TW3F':{'2937': 100000}})

{'portfolio': {'TW3F': {'2937': 100000}, 'SUE01': {'4946': 4498}, 'BETA': {'0050': 6257}}, 'update_time': '2025-05-04 11:36:01'}


In [20]:
# trader.rebalance([(inv.code, 0) for inv in trader.list_inventories()])

In [19]:
trader.list_inventories()

[StockPosition(id=0, code='0050', direction=<Action.Buy: 'Buy'>, quantity=6257, price=159.73, last_price=173.5, pnl=83525.0, yd_quantity=6257, margin_purchase_amount=0, collateral=0, short_sale_margin=0, interest=0),
 StockPosition(id=1, code='2937', direction=<Action.Buy: 'Buy'>, quantity=10000, price=46.25, last_price=44.0, pnl=-24463.0, yd_quantity=10000, margin_purchase_amount=0, collateral=0, short_sale_margin=0, interest=0),
 StockPosition(id=2, code='4946', direction=<Action.Buy: 'Buy'>, quantity=4498, price=111.16, last_price=190.0, pnl=350853.0, yd_quantity=4498, margin_purchase_amount=0, collateral=0, short_sale_margin=0, interest=0)]

## fubon trade

In [12]:
api = FubonAccount()
trader = FubonTrader(api)

portfolio.json loaded from /Users/jianrui/Desktop/Research/Quant/portfolio/fubon_portfolio.json
portfolio.json matches inventories.


In [11]:
# trader.update_portfolio()
trader.portfolio

{'portfolio': {'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}},
 'update_time': '2025-05-06 12:35:34'}

In [27]:
write_portfolio(trader.portfolio, trader.portfolio_path)

portfolio.json saved to /Users/jianrui/Desktop/Research/Quant/portfolio/fubon_portfolio.json


In [5]:
trader.portfolio

{'portfolio': {'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}},
 'update_time': '2025-05-06 12:34:41'}

In [7]:
trader.rebalance({'TEST':{'2881': 100}, 'TEST2':{'2881': 1000}}, by='quantity', freq=1)
# write_portfolio(trader.portfolio)

Current time: 12:35:34
Portfolio is all filled.
portfolio.json saved to /Users/jianrui/Desktop/Research/Quant/portfolio/fubon_portfolio.json


In [6]:
trader.portfolio

{'portfolio': {'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}},
 'update_time': '2025-05-06 12:08:47'}

In [5]:
for order in trader.list_orders():
    trader.cancel_order(order)

In [4]:
trader.list_inventories()

[Inventory {
     date: "2025/05/06",
     account: "217177",
     branch_no: "20210",
     stock_no: "2881",
     order_type: Stock,
     lastday_qty: 0,
     buy_qty: 1000,
     buy_filled_qty: 1000,
     buy_value: 78800,
     today_qty: 1000,
     tradable_qty: 0,
     sell_qty: 0,
     sell_filled_qty: 0,
     sell_value: 0,
     odd: InventoryOdd {
         lastday_qty: 0,
         buy_qty: 100,
         buy_filled_qty: 100,
         buy_value: 7900,
         today_qty: 100,
         tradable_qty: 0,
         sell_qty: 0,
         sell_filled_qty: 0,
         sell_value: 0,
     },
 }]

In [ ]:
def check_portfolio_match_inventories(portfolio:dict):
        agg_json_portfolio = {}
        for p in portfolio['portfolio'].values():
            for stock_id, quantity in p.items():
                if stock_id not in agg_json_portfolio:
                    agg_json_portfolio[stock_id] = quantity
                else:
                    agg_json_portfolio[stock_id] += quantity

        agg_inv_portfolio = {i.code:i.quantity * (-1 if i.direction == 'Sell' else 1)  for i in trader.list_inventories()}
        diff_portfolio = {}
        for k, v in agg_json_portfolio.items():
            if k not in agg_inv_portfolio:
                diff_portfolio[k] = v
            else:
                diff_portfolio[k] = v - agg_inv_portfolio[k]
        diff_portfolio = {k:v for k, v in diff_portfolio.items() if v != 0}
        if diff_portfolio=={}:
            print('portfolio.json matches inventories.')
        else:
            print(f'portfolio.json does not match inventories: {diff_portfolio}!')


AttributeError: 'FubonTrader' object has no attribute 'settlement_is_enough'

In [35]:
trader.portfolio

{'portfolio': {}, 'update_time': '2025-05-05 17:53:20'}

In [6]:
[order for order in trader.list_orders()]

[OrderResult {
     function_type: None,
     date: "2025/05/06",
     seq_no: "02075306775",
     branch_no: "20210",
     account: "217177",
     order_no: "o1287",
     asset_type: 0,
     market: "TAIEX",
     market_type: IntradayOdd,
     stock_no: "2881",
     buy_sell: Buy,
     price_type: Limit,
     price: 78.7,
     quantity: 100,
     time_in_force: ROD,
     order_type: Stock,
     is_pre_order: false,
     status: 30,
     after_price_type: None,
     after_price: 78.7,
     unit: 1,
     after_qty: 0,
     filled_qty: 0,
     filled_money: 0,
     before_qty: None,
     before_price: None,
     user_def: None,
     last_time: "09:50:24.042",
     details: None,
     error_message: None,
 },
 OrderResult {
     function_type: None,
     date: "2025/05/06",
     seq_no: "02075306995",
     branch_no: "20210",
     account: "217177",
     order_no: "o1295",
     asset_type: 0,
     market: "TAIEX",
     market_type: IntradayOdd,
     stock_no: "2881",
     buy_sell: Buy,
 

In [10]:
dt.datetime.strptime(f"{order.date} {order.last_time}", "%Y/%m/%d %H:%M:%S.%f")


datetime.datetime(2025, 5, 6, 10, 2, 27, 822000)

In [85]:
trader.list_inventories()

[Inventory {
     date: "2025/05/06",
     account: "217177",
     branch_no: "20210",
     stock_no: "2881",
     order_type: Stock,
     lastday_qty: 0,
     buy_qty: 1000,
     buy_filled_qty: 1000,
     buy_value: 78800,
     today_qty: 1000,
     tradable_qty: 0,
     sell_qty: 0,
     sell_filled_qty: 0,
     sell_value: 0,
     odd: InventoryOdd {
         lastday_qty: 0,
         buy_qty: 100,
         buy_filled_qty: 100,
         buy_value: 7900,
         today_qty: 100,
         tradable_qty: 0,
         sell_qty: 0,
         sell_filled_qty: 0,
         sell_value: 0,
     },
 }]

In [93]:
trader.portfolio['portfolio']

{'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}}

In [90]:
def check_portfolio_difference(new_portfolio:Dict[str, List[Dict[str, Union[int, float]]]], by:Literal['money', 'quantity']='money'):
    difference_portfolio = {}
    if by == 'money':
        for strategy, positions in new_portfolio.items():
            difference_portfolio[strategy] = []
            current_strategy = trader.portfolio['portfolio'].get(strategy, {})
            
            def process_money(stock_id, money):
                inv_money = current_strategy.get(stock_id, 0) * next((sum([inv.buy_value, inv.odd.buy_value])/sum([inv.today_qty, inv.odd.today_qty]) for inv in trader.list_inventories() if inv.stock_no == stock_id), 0)
                position = Position().from_money(stock_id, money - inv_money)
                if not position.empty:
                    return position
                return None
            
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(process_money, stock_id, money) for stock_id, money in positions.items()]
                for future in as_completed(futures):
                    position = future.result()
                    if position:
                        difference_portfolio[strategy].append(position)   
    elif by == 'quantity':
        for strategy, positions in new_portfolio.items():
            difference_portfolio[strategy] = []
            current_strategy = trader.portfolio['portfolio'].get(strategy, {})
            
            def process_quantity(stock_id, quantity):
                inv_quantity = current_strategy.get(stock_id, 0)
                position = Position().from_quantity(stock_id, quantity - inv_quantity)
                if not position.empty:
                    return position
                return None
            
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(process_quantity, stock_id, quantity) for stock_id, quantity in positions.items()]
                for future in as_completed(futures):
                    position = future.result()
                    if position:
                        difference_portfolio[strategy].append(position)
    return difference_portfolio


In [101]:
check_portfolio_difference({'TEST2':{'2881': 78800}}, by='money')

{'TEST2': []}

In [21]:
traded_orders

{('TEST', '2881'): 100, ('TEST2', '2881'): 1000}

In [82]:
order_status = {
    0: '預約單',
    4: '系統將委託送往後台',
    9: '連線逾時',
    10: '委託成功',
    30: '未成交刪單成功',
    40: '部分成交，剩餘取消',
    50: '完全成交',
    90: '失敗',
}
current_portfolio = trader.portfolio['portfolio'].copy()
last_update_time = dt.datetime.strptime(trader.portfolio['update_time'], '%Y-%m-%d %H:%M:%S')
traded_orders={
    (order.user_def, order.stock_no):\
    order.filled_qty * \
    (-1 if order.buy_sell=='Sell' else 1)\
    for order in trader.list_orders() if \
    (order_status[order.status] in ['完全成交', '部分成交，剩餘取消']) & (dt.datetime.strptime(f"{order.date} {order.last_time}", "%Y/%m/%d %H:%M:%S.%f") > last_update_time)
}

for k, v in traded_orders.items():
    if not current_portfolio.get(k[0]):
        current_portfolio[k[0]]={}
    if not current_portfolio[k[0]].get(k[1]):
        current_portfolio[k[0]][k[1]] = v
    else:
        current_portfolio[k[0]][k[1]] += v
trader.portfolio['portfolio'] = current_portfolio
trader.portfolio['update_time'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

trader.portfolio

{'portfolio': {'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}},
 'update_time': '2025-05-06 11:46:37'}

In [83]:
trader.portfolio #= {'portfolio': {'TEST': {}}, 'update_time': '2025-05-05 17:53:20'}

{'portfolio': {'TEST': {'2881': 100}, 'TEST2': {'2881': 1000}},
 'update_time': '2025-05-06 11:46:37'}

In [74]:
trader.portfolio =read_portfolio(trader.portfolio_path) #['portfolio']

portfolio.json loaded from /Users/jianrui/Desktop/Research/Quant/portfolio/fubon_portfolio.json


In [11]:
sum([s.total_settlement_amount if s.total_settlement_amount else 0 for s in trader.get_settlement().details])

[0, 0, 0]

In [7]:
sum([settlement.total_settlement_amount or 0 for settlement in trader.get_settlement().details])

0

In [91]:
from fubon_neo.sdk import FubonSDK, Order
from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction


In [92]:
sdk = FubonSDK()
account = sdk.login("A127945492", "ryankan209", "/Users/jianrui/Desktop/API_pfx/fubon.pfx", "ryankan209").data[0]
account

Account {
    name: "簡叡",
    branch_no: "20210",
    account: "217177",
    account_type: "stock",
}

In [79]:
sdk.accounting.query_settlement(account, '3d').data

SettlementData {
    account: AccountRes {
        branch_no: "20210",
        account: "217177",
    },
    details: [
        Settlement {
            date: "2025/04/29",
            settlement_date: None,
            buy_value: None,
            buy_fee: None,
            buy_settlement: None,
            buy_tax: None,
            sell_value: None,
            sell_fee: None,
            sell_settlement: None,
            sell_tax: None,
            total_bs_value: None,
            total_fee: None,
            total_tax: None,
            total_settlement_amount: None,
            currency: None,
        },
        Settlement {
            date: "2025/04/30",
            settlement_date: None,
            buy_value: None,
            buy_fee: None,
            buy_settlement: None,
            buy_tax: None,
            sell_value: None,
            sell_fee: None,
            sell_settlement: None,
            sell_tax: None,
            total_bs_value: None,
            total_fe